# I will train a faster-rcnn based on vgg16, because I only have a GTX980M with only 4GB memoryon my dell laptop.

## import libraries

In [ ]:
#%matplotlib inline
from __future__ import division
from __future__ import print_function
import _init_paths
import os
import numpy as np
from caffe2.python import workspace, model_helper, core

from modeling import model_builder, VGG16, rpn_heads

from core.config import merge_cfg_from_file, cfg

from roi_data.loader import RoIDataLoader
from datasets.roidb import combined_roidb_for_training
from utils.training_stats import TrainingStats

workspace.GlobalInit(['caffe2', '--caffe2_log_level=-1', '--caffe2_gpu_memory_tracking=1'])
merge_cfg_from_file('./e2e_faster_rcnn_vgg16_pascal2007.yaml')
output_dir = os.path.join(os.getcwd(), 'output')

## create model

In [ ]:
import utils.c2
utils.c2.import_contrib_ops()
utils.c2.import_detectron_ops()

#调用 model_builder.create 函数创建model
train_model = model_builder.create(cfg.MODEL.TYPE, train=True)
workspace.RunNetOnce(train_model.param_init_net)

#添加数据dataset
"""Load the training dataset and attach the training inputs to the model."""
print('Loading dataset: {}'.format(cfg.TRAIN.DATASETS))
print('Loading proposal:{}'.format(cfg.TRAIN.PROPOSAL_FILES))

roidb = combined_roidb_for_training(cfg.TRAIN.DATASETS, cfg.TRAIN.PROPOSAL_FILES)
model_builder.add_training_inputs(train_model, roidb=roidb)

#把网络村到文件里，并欣赏一下创建的网络
with open('./net.pbtxt', 'w') as fid:
    fid.write(str(train_model.net.Proto()))
with open('./param_init_net.pbtxt', 'w') as fid:
    fid.write(str(train_model.param_init_net.Proto()))

In [ ]:
workspace.CreateNet(train_model.net, overwrite=True)
# Start loading mini-batches and enqueuing blobs
train_model.roi_data_loader.register_sigint_handler()
train_model.roi_data_loader.start(prefill=True)


In [ ]:
# Save model weights in a dictonay and pickle them to a file.
# copy from util/net.py
def save_weights(weights_file, model):
    blobs = {}
    for param in model.params:
        print(param)
        param_name = str(param)
        if param_name not in blobs:
            blobs[param_name] = workspace.FetchBlob(param_name)
    cfg_yaml = yaml.dump(cfg)
    with open(weights_file, 'wb') as fid:
        pickle.dump(dict(blobs=blobs, cfg=cfg_yaml), fid, pickle.HIGHTEST_PROTOCOL)
    

In [ ]:
print(train_model.losses)

def sum_losses(model):
    loss = 0.0
    for k in train_model.losses:
        loss += workspace.FetchBlob('gpu_{}/{}'.format(0,k))

In [ ]:
#开始训练
#ITER = cfg.SOLVER.MAX_ITER
ITER = 10
losses = np.zeros(ITER)
metrics = np.zeros(ITER)

#print(workspace.Blobs())    
for cur_iter in range(ITER):
    workspace.RunNet(train_model.net.Proto().name)
    
    #Got loss
    losses[cur_iter] = sum_losses(train_model)
    
    if (cur_iter + 1) % cfg.TRAIN.SNAPSHOT_ITERS == 0 and cur_iter > start_iter:
        print("Iter:{}, loss:{:0.8f},metrics:{}".format(cur_iter, losses[cur_iter], metrics[cur_iter]))
        weights_file =os.path.join(output_dir, 'model_iter{}.pkl'.format(cur_iter))
        save_weights(weights_file, train_model)
    
train_model.roi_data_loader.shutdown()

In [ ]:
print(losses)